<a href="https://colab.research.google.com/github/cpng123/Database-Advanced-Data-Techniques/blob/main/SPARQL%20Hand-On%20Lab%20-%20Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction to RDF and SPARQL
RDF (Resource Description Framework) is a standard model for data interchange on the web. SPARQL is a query language for RDF. This lab will introduce the basics of RDF, how to create RDF data in Turtle format, and how to query it using SPARQL.


# 2. Setting Up the Environment

First, we need to install the `rdflib` library, which provides tools for working with RDF data in Python.


In [1]:
# Install rdflib library
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.0/567.0 kB 11.4 MB/s eta 0:00:00


We also need to import necessary modules.

In [2]:
# Import necessary modules
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import FOAF, XSD, DC

# Use Case 1: Basic RDF and SPARQL

# 3. Creating and Saving RDF Turtle Document

We will create a simple RDF Turtle document and save it using the `%%writefile` magic cell.


In [3]:
%%writefile data.ttl
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .

ex:john a foaf:Person ;
    foaf:name "John Doe"^^xsd:string ;
    foaf:knows ex:jane, ex:bob ;
    foaf:age "30"^^xsd:integer ;
    foaf:mbox <mailto:john@example.org> .

ex:jane a foaf:Person ;
    foaf:name "Jane Smith"^^xsd:string ;
    foaf:knows ex:john ;
    foaf:age "28"^^xsd:integer ;
    foaf:mbox <mailto:jane@example.org> ;
    dc:creator ex:book1 .

ex:bob a foaf:Person ;
    foaf:name "Bob Johnson"^^xsd:string ;
    foaf:knows ex:john ;
    foaf:age "35"^^xsd:integer ;
    foaf:mbox <mailto:bob@example.org> .

ex:book1 a dc:Book ;
    dc:title "Learning RDF"^^xsd:string ;
    dc:creator ex:jane ;
    dc:date "2021-01-01"^^xsd:date .

Writing data.ttl


# 4. Loading RDF Turtle Document

We can load RDF data from the Turtle document directly into an RDF graph.

In [4]:
# Create a new RDF graph
g = Graph()

# Load the Turtle data from the file
g.parse('data.ttl', format='turtle')

# Verify the graph contents
for stmt in g:
    print(stmt)

(rdflib.term.URIRef('http://example.org/book1'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://purl.org/dc/elements/1.1/Book'))
(rdflib.term.URIRef('http://example.org/book1'), rdflib.term.URIRef('http://purl.org/dc/elements/1.1/creator'), rdflib.term.URIRef('http://example.org/jane'))
(rdflib.term.URIRef('http://example.org/book1'), rdflib.term.URIRef('http://purl.org/dc/elements/1.1/date'), rdflib.term.Literal('2021-01-01', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')))
(rdflib.term.URIRef('http://example.org/bob'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/knows'), rdflib.term.URIRef('http://example.org/john'))
(rdflib.term.URIRef('http://example.org/bob'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/age'), rdflib.term.Literal('35', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://example.org/jane'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-r

# 5. Querying RDF Data with SPARQL

We will use SPARQL to query the RDF data we loaded.


**a. Querying all people and their names:**

In [5]:
# Define a SPARQL query
query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>
SELECT ?person ?name
WHERE {
  ?person a foaf:Person .
  ?person foaf:name ?name .
}
"""

# Execute the query on the loaded graph
qres = g.query(query)

# Print the results
for row in qres:
    print(f"{row.person} is named {row.name}")

http://example.org/john is named John Doe
http://example.org/jane is named Jane Smith
http://example.org/bob is named Bob Johnson


**b. Querying relationships (who knows whom):**

In [13]:
# Define a SPARQL query for relationships
query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>
SELECT ?person1_name ?person2_name ?person1_age ?person2_age
WHERE {
  ?person1 foaf:knows ?person2 .
  ?person1 foaf:name ?person1_name .
  ?person2 foaf:name ?person2_name .
  ?person1 foaf:age ?person1_age .
  ?person2 foaf:age ?person2_age .
}
"""

# Execute the query on the loaded graph
qres = g.query(query)

# Print the results
for row in qres:
    print(f"[{row.person1_name} age {row.person1_age}] knows [{row.person2_name} age {row.person2_age}]")

[John Doe age 30] knows [Jane Smith age 28]
[John Doe age 30] knows [Bob Johnson age 35]
[Jane Smith age 28] knows [John Doe age 30]
[Bob Johnson age 35] knows [John Doe age 30]


# 6. Advanced SPARQL Queries

Let's explore some advanced SPARQL queries.

**a. Querying with optional properties:**

In [14]:
# Define a SPARQL query with an optional property
query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>
SELECT ?person ?name ?mbox
WHERE {
  ?person a foaf:Person .
  ?person foaf:name ?name .
  OPTIONAL { ?person foaf:mbox ?mbox . }
}
"""

# Execute the query on the loaded graph
qres = g.query(query)

# Print the results
for row in qres:
    print(f"{row.person} {row.name} {row.mbox}")

http://example.org/john John Doe mailto:john@example.org
http://example.org/jane Jane Smith mailto:jane@example.org
http://example.org/bob Bob Johnson mailto:bob@example.org


**b. Querying with filters:**

In [15]:
# Define a SPARQL query with a filter
query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>
SELECT ?person ?name
WHERE {
  ?person a foaf:Person .
  ?person foaf:name ?name .
  FILTER regex(?name, "John", "i")
}
"""

# Execute the query on the loaded graph
qres = g.query(query)

# Print the results
for row in qres:
    print(f"{row.person} {row.name}")

http://example.org/john John Doe
http://example.org/bob Bob Johnson


**c. Querying books and their creators:**

In [16]:
# Define a SPARQL query to get books and their creators
query = """
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX ex: <http://example.org/>
SELECT ?book ?title ?creator
WHERE {
  ?book a dc:Book .
  ?book dc:title ?title .
  ?book dc:creator ?creator .
}
"""

# Execute the query on the loaded graph
qres = g.query(query)

# Print the results
for row in qres:
    print(f"{row.book} {row.title} {row.creator}")

http://example.org/book1 Learning RDF http://example.org/jane


# Use Case 2: Advanced RDF and SPARQL

# 7. Creating and Saving a More Complex RDF Turtle Document

We will create a more complex RDF Turtle document and save it using the `%%writefile` magic cell.

In [17]:
%%writefile complex_data.ttl
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .

ex:john a foaf:Person ;
    foaf:name "John Doe"^^xsd:string ;
    foaf:knows ex:jane, ex:bob ;
    foaf:age "30"^^xsd:integer ;
    foaf:mbox <mailto:john@example.org> ;
    ex:hasPet ex:spot .

ex:jane a foaf:Person ;
    foaf:name "Jane Smith"^^xsd:string ;
    foaf:knows ex:john, ex:alice ;
    foaf:age "28"^^xsd:integer ;
    foaf:mbox <mailto:jane@example.org> ;
    dc:creator ex:book1 .

ex:bob a foaf:Person ;
    foaf:name "Bob Johnson"^^xsd:string ;
    foaf:knows ex:john ;
    foaf:age "35"^^xsd:integer ;
    foaf:mbox <mailto:bob@example.org> .

ex:alice a foaf:Person ;
    foaf:name "Alice Brown"^^xsd:string ;
    foaf:knows ex:jane ;
    foaf:age "25"^^xsd:integer ;
    foaf:mbox <mailto:alice@example.org> .

ex:book1 a dc:Book ;
    dc:title "Learning RDF"^^xsd:string ;
    dc:creator ex:jane ;
    dc:date "2021-01-01"^^xsd:date .

ex:spot a ex:Pet ;
    ex:name "Spot"^^xsd:string ;
    ex:type "Dog"^^xsd:string ;
    ex:owner ex:john .

Writing complex_data.ttl


# 8. Loading the More Complex RDF Turtle Document

We can load RDF data from the Turtle document directly into an RDF graph.


In [18]:
# Create a new RDF graph
g_complex = Graph()

# Load the Turtle data from the file
g_complex.parse('complex_data.ttl', format='turtle')

# Verify the graph contents
for stmt in g_complex:
    print(stmt)

(rdflib.term.URIRef('http://example.org/alice'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/name'), rdflib.term.Literal('Alice Brown', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://example.org/bob'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/knows'), rdflib.term.URIRef('http://example.org/john'))
(rdflib.term.URIRef('http://example.org/bob'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/age'), rdflib.term.Literal('35', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://example.org/jane'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Person'))
(rdflib.term.URIRef('http://example.org/jane'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/knows'), rdflib.term.URIRef('http://example.org/alice'))
(rdflib.term.URIRef('http://example.org/john'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/mbox'), rdflib.term.

# 9. Querying the More Complex RDF Data with SPARQL

We will use SPARQL to query the more complex RDF data we loaded.


**a. Querying people and their pets:**

In [19]:
# Define a SPARQL query to get people and their pets
query = """
PREFIX ex: <http://example.org/>
SELECT ?person ?pet ?petName ?petType
WHERE {
  ?person ex:hasPet ?pet .
  ?pet ex:name ?petName .


 ?pet ex:type ?petType .
}
"""

# Execute the query on the loaded graph
qres = g_complex.query(query)

# Print the results
for row in qres:
    print(f"{row.person} {row.pet} {row.petName} {row.petType}")

http://example.org/john http://example.org/spot Spot Dog


**b. Querying people who know others with pets:**


In [34]:
# Define a SPARQL query to get people and their pets
query = """
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?person_name
WHERE {
  ?friend ex:hasPet ?pet .
  ?person foaf:knows ?friend .
  ?person foaf:name ?person_name .
}
"""

# Execute the query on the loaded graph
qres = g_complex.query(query)

# Print the results
for row in qres:
    print(f"{row.person_name}")

Jane Smith
Bob Johnson


**a. Extract titles of all books (list example):**


In [35]:
# Define a SPARQL query to get people who know others with pets
query = """
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?person ?knownPerson ?petName
WHERE {
  ?person foaf:knows ?knownPerson .
  ?knownPerson ex:hasPet ?pet .
  ?pet ex:name ?petName .
}
"""

# Execute the query on the loaded graph
qres = g_complex.query(query)

# Print the results
for row in qres:
    print(f"{row.person} {row.knownPerson} {row.petName}")

http://example.org/jane http://example.org/john Spot
http://example.org/bob http://example.org/john Spot


**c. Querying people who are creators and their books:**

In [36]:
# Define a SPARQL query to get people who are creators and their books
query = """
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX ex: <http://example.org/>
SELECT ?person ?book ?title
WHERE {
  ?book dc:creator ?person .
  ?book dc:title ?title .
}
"""

# Execute the query on the loaded graph
qres = g_complex.query(query)

# Print the results
for row in qres:
    print(f"{row.person} {row.book} {row.title}")

http://example.org/jane http://example.org/book1 Learning RDF


# 10. Conclusion

RDF and SPARQL are powerful tools for representing and querying data on the web. In this lab, we've covered two use cases: the basics of creating RDF data, saving it to a Turtle document, loading RDF data from a Turtle document, and querying it using SPARQL; and a more complex example with advanced RDF structures and queries. You can further explore RDF and SPARQL to handle more complex data and queries.
